In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0'


import spacy
nlp = spacy.load("fr_core_news_lg")


In [ ]:
import sys
sys.path.append('..')
from extract_corpus import get_coref_corpus

# on utilise pour les adjectifs le corpus avec la coreference
# car sinon on va trouver que par ex l'adjective "gentille" se rapporte a "Elle" dans la phrase
# "Mme Tiger est doctorante. Elle est gentille".
# ce qui n'est pas super utile
# alors que le corpus coref a fait une passe sur toutes les phrases du dossier data pour transformer les phrases en 
# "Mme Tiger est doctorante. Mme Tiger est gentille".
# et donc on trouvera que "gentille" se rapporte a "Mme Tiger"
corpus = get_coref_corpus()

In [ ]:
# petit script pour voir ce qui est contenu dans chaque token
doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")
for tok in doc:
  print(f"{tok.text:<10} {tok.dep_:<10} {tok.pos_} {tok.head}")

In [ ]:

def compute_adjectives_subjects(doc):
    # ici doc, correspond au texte d'une annee analyse par spacy 
    # cf: 
    # year_text = "\n".join([chunk.text for chunk in corpus[year]]) )
    # doc = nlp(year_text)


    # cette fonction va remplir l'objet global_adjectives_subjects
    # ca se passe en 2 parties
    # 1) on va trouver tous les adjectivs et sujets pour ce doc (donc cette annee)
    # 2) on va completer la variable global_adjectives_subjects qui contiendra tous les adjectivs et sujets cumules sur toutes les annees
    

    # 1) on instancie local_adj_subj : un objet qui s'occupera de contenir
    # pour chaque sujet une liste de ses adjectifs POUR L'ANNEE EN COURS (pour le doc)
    local_adj_subj = {}
    # dans un premier temps, on cherche dans le doc (qui correspond au texte d'une annee) toutes les Entitees qui sont des personnes (Mme Tiger,...)
    for ent in doc.ents:
        if ent.label_ == "PER":
            # Si on trouve une entitee qui est une personne on dit que local_adj_subj[l'entitee] est un [] tableau vide (on a trouve un sujet, on trouvera ses adjectifs apres)
            local_adj_subj[ent] = []
            
    # Donc a ce niveau on a rempli local_adj_subj de la facon suivante : {PERSONNE1: [], PERSONNE2: [], ...}
    # ou PERSONNE 1, PERSONNE 2, sont du type Spacy.Span (https://spacy.io/api/span), ce qui veut dire qu'elles contiennent plusieurs tokens (ou un mais c'est quand meme une liste d'un element)

    # maintenant on va boucler sur nos sujets (ent) et leus adjectifs associes (qui ici sont juste une liste vide : []) 
    # puis justement essayer de trouver les adjectifs
    for ent, adjs in local_adj_subj.items():
        # ici ent est donc une Span (une liste de tokens, ex [Mme, Tiger])
        for token in ent:
            # pour chaque token (Mme, puis Tiger) on va verifier que son head (ce a quoi il est ratache, n'est pas un ADJectif) (tu peux regarder l'attribut head dans https://spacy.io/api/token#attributes)
            # et pour te familiariser avec la facon dont on cherche les infos avec spacy : https://spacy.io/usage/linguistic-features
            if token.head.pos_ == "ADJ":
                # si le head d'une partie du sujet est un ADJectif, on l'ajoute aux adjs(ectifs) du sujet
                adjs.append(token.head)
    
    # Donc a ce niveau on a rempli local_adj_subj de la facon suivante : {PERSONNE1: [adj1, adj2,], PERSONNE2: [adj3,adj4], ...} 
    # pour l'annee actuelle (car on a rempli local_adj_subj et non pas l'objet global global_adjectives_subjects)

    
    # 2) on met à jour l'objet global global_adjectives_subjects qui contiendra tous les adjectifs et sujets cumulés sur TOUTES LES ANNÉES
    # Pour chaque sujet / adjectifs trouves dans l'annee
    for subject, adjs in local_adj_subj.items():
        # Si le sujet n'existe pas encore dans le dictionnaire global (global_adjectives_subjects)
        if subject.text not in global_adjectives_subjects:
            # On crée une nouvelle entrée avec une liste vide
            global_adjectives_subjects[subject.text] = []
        # On ajoute les adjectifs de cette année à la liste existante
        global_adjectives_subjects[subject.text].extend(adjs)


# on defini un objet global_adjectives_subjects qui est ammene a contenir, 
# pour chaque sujet, la liste d'adjectifs associes sur toutes les annees:
# ex  global_adjectives_subjects sera egal a ~ {"Mme TIGER": ["gentille", ...],}
global_adjectives_subjects = {}


# pour chaque annee du corpus de coref
# on va recuperer le texte de l'annee
# puis lancer la fonction compute_adjectives_subjects sur le doc spacy du texte 
for year in corpus:
    print("YEAR = ", year)

    #le but c'est de concatener le texte de chaque chunk de 1814
    # comme dans les documents txt d'origine, on a ajoute une Chunk par ligne (cf process_document -> for line in text.split('\n'):),
    # pour retomber sur nos pattes, et avoir la meme structure de document que le texte d'origine, on joint les Chunk (lignes) par un retour a la ligne (\n)
    year_text = "\n".join([chunk.text for chunk in corpus[year]]) #le but c'est de concatener le texte de chaque chunk de 1917

    doc = nlp(year_text)
    compute_adjectives_subjects(doc)



In [19]:
# ici simplement on affiche les sujets et leurs adjectifs
for subject, adjs in global_adjectives_subjects.items():
    if (len(adjs)>0):
        print(f"{subject} -> {adjs}")


Silvain -> [excessif]
M. -> [supérieur, prêts]
M. Raphaël Duflos -> [douzième]
Mme -> [première]
M. de Max -> [relative]
Mme Piérat -> [douzième, douzième]
M. Mouché -> [agréable]
M. Dessonnes -> [douzième, relatif, Française]
M. Lucien Pallez -> [pendante]
M. Fontaine -> [nouvelles]
M. Grand -> [prêts]
M. Brunot -> [douzième]
M. Croué -> [douzième]
M. Bernard -> [douzième]
Mlle Delvair -> [douzième]
Mme Silvain -> [douzième, relatives, relative, relative]
Mlle Roch -> [douzième]
Mlle Valpreux -> [prêts]
M. Devred -> [droit]
Elvire du Cid -> [tels, tels]
Doris d’Iphigénie -> [inutile]
Mme Lamorlette -> [impossible]
M. Luguet -> [libre]
M. P. Gavault -> [autre]
Mlle Géniat -> [disponibles]
G. Dubot -> [Dernière]
M. Henry Roy -> [autre, autre]
M. Albert Carré ancien -> [agréable]
meai -> [premier]
MM. Granval -> [relatives]


In [45]:
# une facon plus pratique de rechercher dans les resultats

from ipywidgets import widgets
from IPython.display import display, HTML

# Create text widgets for filtering subjects and adjectives
filter_subject = widgets.Text(
    value='',
    placeholder='Filter subjects',
    description='Subject:',
    disabled=False
)

filter_adj = widgets.Text(
    value='',
    placeholder='Filter adjectives',
    description='Adjective:',
    disabled=False
)

# Create an Output widget to display the table
table_output = widgets.Output()

# Function to filter and display the table
def filter_table(change):
    with table_output:
        table_output.clear_output(wait=True)
        subject_text = filter_subject.value.lower()
        adj_text = filter_adj.value.lower()
        
        rows = []
        for subject, adjs in global_adjectives_subjects.items():
            if subject_text in subject.lower():
                matching_adjs = [adj for adj in adjs if adj_text in adj.text.lower()]
                if matching_adjs:
                    adj_string = ', '.join([adj.text for adj in matching_adjs])
                    row = f"<tr><td style='text-align:left;'><b>{subject}</b></td><td style='text-align:left;'>{adj_string}</td></tr>"
                    rows.append(row)
        
        table = "<table>" + "".join(rows) + "</table>"
        display(HTML(table))

# Display the filter widgets and create a container for the table
display(widgets.HBox([filter_subject, filter_adj]))
display(table_output)

# Bind the filter function to both text widgets
filter_subject.observe(filter_table, names='value')
filter_adj.observe(filter_table, names='value')

# Initial table display
filter_table({'new': ''})

Output()